In [5]:
# Import Pytorch Libraries
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models

In [6]:
# Import other libraries
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
# Select Device
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
# extract data from Zipfile
import zipfile
with zipfile.ZipFile('animal_data.zip','r') as z:
    z.extractall('animal_data')

In [9]:
#preprocess the image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [10]:
# Making dataset
dataset=datasets.ImageFolder('animal_data/animal_data',transform=transform)
num_classes=len(dataset.classes)
dataset.classes

train_size=int(0.8*len(dataset))
test_size=len(dataset)-train_size
train_dataset,test_dataset=random_split(dataset,[train_size,test_size])
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False)

In [11]:
# Function for training model
def train_model(model,criterion,optimizer,epochs):
    model.train()
    running_loss=0.0
    for i in range(epochs):
        running_loss=0.0
        for images, labels in train_loader:
           images,labels=images.to(device),labels.to(device)
           
           optimizer.zero_grad()
           output=model(images)
           loss=criterion(output,labels)
           loss.backward()
           optimizer.step()

           running_loss+=loss.item()


        print(f"Epoch [{i+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

In [12]:
# Function for evaluating model
def eval_model(model):
    y_true,y_pred=[],[]
    model.eval()
    with torch.no_grad():
        for images , labels in test_loader:
            images,labels=images.to(device),labels.to(device)
            output=model(images)
            pred=torch.argmax(output,dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(pred.cpu().numpy())
            
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted')
    rec = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    return acc, prec, rec, f1

## First Model

In [13]:
# Baseline CNN
# create the model
class baselineCNN(nn.Module):
    def __init__(self,num_classes):
        super().__init__()
        self.features=nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.output=nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 56 * 56, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self,x):
        x=self.features(x)
        x=self.output(x)
        return x

In [14]:
# Create an object for model and selecting loss function and optimizer
model1=baselineCNN(num_classes).to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model1.parameters(),lr=0.001)


In [15]:
#train the model
train_model(model1,criterion,optimizer,10)

Epoch [1/10], Loss: 3.5305
Epoch [2/10], Loss: 2.5760
Epoch [3/10], Loss: 2.3705
Epoch [4/10], Loss: 2.1846
Epoch [5/10], Loss: 1.8397
Epoch [6/10], Loss: 1.3927
Epoch [7/10], Loss: 0.8588
Epoch [8/10], Loss: 0.5923
Epoch [9/10], Loss: 0.4558
Epoch [10/10], Loss: 0.4326


In [16]:
# Evaluate the model
m1_metrics=eval_model(model1)
m1_metrics

(0.5886889460154242,
 0.6576723609912556,
 0.5886889460154242,
 0.5934450795470319)

# Top 3 models

## GoogleNet CNN

In [17]:
# Loading the model
googlenet = models.googlenet(
    weights=models.GoogLeNet_Weights.IMAGENET1K_V1  
)

for param in googlenet.parameters():
    param.requires_grad = False

googlenet.fc = nn.Sequential(
    nn.Linear(googlenet.fc.in_features, 256),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    nn.Linear(256, num_classes)
)

In [18]:
# Create an object for model and selecting loss function and optimizer
model2 = googlenet.to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model2.parameters(),lr=0.001)

In [19]:
# Train the model
train_model(model2,criterion,optimizer,10)

Epoch [1/10], Loss: 1.8752
Epoch [2/10], Loss: 0.7353
Epoch [3/10], Loss: 0.4767
Epoch [4/10], Loss: 0.3388
Epoch [5/10], Loss: 0.2958
Epoch [6/10], Loss: 0.2633
Epoch [7/10], Loss: 0.2240
Epoch [8/10], Loss: 0.2130
Epoch [9/10], Loss: 0.1711
Epoch [10/10], Loss: 0.1597


In [20]:
# Evaluate the model
m2_metrics=eval_model(model2)
m2_metrics

(0.9537275064267352,
 0.9585320001059283,
 0.9537275064267352,
 0.9540366936350758)

## MobileNetV2

In [21]:
# Loading the model
mobilenet = models.mobilenet_v2(
    weights=models.MobileNet_V2_Weights.IMAGENET1K_V1
)

for param in mobilenet.features.parameters():
    param.requires_grad = False

mobilenet.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(mobilenet.last_channel, num_classes)
)

# Create an object for model and selecting loss function and optimizer
model3 = mobilenet.to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model3.parameters(),lr=0.001)



In [22]:
# Train the model
train_model(model3, criterion, optimizer, epochs=10)

Epoch [1/10], Loss: 1.6394
Epoch [2/10], Loss: 0.6306
Epoch [3/10], Loss: 0.4222
Epoch [4/10], Loss: 0.3251
Epoch [5/10], Loss: 0.2709
Epoch [6/10], Loss: 0.2340
Epoch [7/10], Loss: 0.1971
Epoch [8/10], Loss: 0.1924
Epoch [9/10], Loss: 0.1626
Epoch [10/10], Loss: 0.1482


In [23]:
# Evaluate the model
m3_metrics=eval_model(model3)
m3_metrics

(0.9717223650385605,
 0.9732963943706738,
 0.9717223650385605,
 0.9719904876590578)

## ResNet CNN

In [24]:
# Making the model
resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

for param in resnet.parameters():
    param.requires_grad = False

resnet.fc = nn.Sequential(
    nn.Linear(resnet.fc.in_features, 256),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    nn.Linear(256, num_classes)
)
 

In [25]:
# Create an object for model and selecting loss function and optimizer
model4 = resnet.to(device)
optimizer = torch.optim.Adam(model4.parameters(),lr=0.001)
criterion=nn.CrossEntropyLoss()

# Train the model
train_model(model4, criterion, optimizer, epochs=10)

# Evaluate the model
m4_metrics = eval_model(model4)
m4_metrics

Epoch [1/10], Loss: 1.6010
Epoch [2/10], Loss: 0.5395
Epoch [3/10], Loss: 0.3395
Epoch [4/10], Loss: 0.2765
Epoch [5/10], Loss: 0.2071
Epoch [6/10], Loss: 0.2016
Epoch [7/10], Loss: 0.1850
Epoch [8/10], Loss: 0.1361
Epoch [9/10], Loss: 0.1445
Epoch [10/10], Loss: 0.1177


(0.9640102827763496,
 0.9666071699203057,
 0.9640102827763496,
 0.9643780474168593)

# Final Result

In [ ]:
# Making a dataframe for storing the final evaluation of all the models
results = pd.DataFrame({
    'Model': ['Baseline CNN', 'GoogleNet', 'MobileNetV2', 'ResNet50'],
    'Accuracy': [m1_metrics[0], m2_metrics[0], m3_metrics[0], m4_metrics[0]],
    'Precision': [m1_metrics[1], m2_metrics[1], m3_metrics[1], m4_metrics[1]],
    'Recall': [m1_metrics[2], m2_metrics[2], m3_metrics[2], m4_metrics[2]],
    'F1 Score': [m1_metrics[3], m2_metrics[3], m3_metrics[3], m4_metrics[3]]
})

results


,Model,Accuracy,Precision,Recall,F1 Score
0,Baseline CNN,0.588689,0.657672,0.588689,0.593445
1,GoogleNet,0.953728,0.958532,0.953728,0.954037
2,MobileNetV2,0.971722,0.973296,0.971722,0.971990
3,ResNet50,0.964010,0.966607,0.964010,0.964378
